In [1]:
import os
import shutil
import json
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.env import PettingZooEnv
from ray.tune.registry import register_env
from spoiled_broth.rl.game_env import GameEnv
from pathlib import Path
import supersuit as ss
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
rl_model = 'PPO'
map_nr = 1

# Define reward weights
reward_weights = {
    "ai_rl_1": (0.5, 0.5),  # agent 1 is full cooperative
    "ai_rl_2": (0.9, 0.1),  # agent 2 is less cooperative
}

In [3]:
base_save_dir = f"/data/samuel_lozano/cooked/saved_models/map_{map_nr}/"
Path(base_save_dir).mkdir(parents=True, exist_ok=True)

In [4]:
def env_creator(config):
    # Your existing GameEnv class goes here
    return GameEnv(**config)

#def env_creator_2(config):
#    env = GameEnv(reward_weights=reward_weights, map_nr=map_nr)
#    env = ss.pad_observations_v0(env)
#    env = ss.pad_action_space_v0(env)
#    return PettingZooEnv(env)

In [5]:
# Initialize Ray
os.environ["RAY_TMPDIR"] = "/data/samuel_lozano/tmp_ray/"
ray.shutdown()
ray.init(ignore_reinit_error=True, _temp_dir=os.environ["RAY_TMPDIR"])

# Register the environment with RLLib
register_env("spoiled_broth", lambda config: ParallelPettingZooEnv(env_creator(config)))


2025-05-18 23:05:16,592	INFO worker.py:1888 -- Started a local Ray instance.


(MultiAgentEnvRunner pid=1607652) Weighted rewards:
(MultiAgentEnvRunner pid=1607652) 
(MultiAgentEnvRunner pid=1607652) Agent 1: (0.5, 0.5)
(MultiAgentEnvRunner pid=1607652) 
(MultiAgentEnvRunner pid=1607652) Agent 2: (0.9, 0.1)
(MultiAgentEnvRunner pid=1607652) 


(MultiAgentEnvRunner pid=1607652) 2025-05-18 23:05:28,960	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!


(MultiAgentEnvRunner pid=1607667) 
(MultiAgentEnvRunner pid=1607667) 
(MultiAgentEnvRunner pid=1607667) 
(MultiAgentEnvRunner pid=1607667) Agent delivered item, new score: 1
(MultiAgentEnvRunner pid=1607652) Agent delivered item, new score: 1
(MultiAgentEnvRunner pid=1607652) Agent delivered item, new score: 2
(MultiAgentEnvRunner pid=1607652) Agent delivered item, new score: 3
(MultiAgentEnvRunner pid=1607652) Agent delivered item, new score: 1
(MultiAgentEnvRunner pid=1607652) Agent delivered item, new score: 1
(MultiAgentEnvRunner pid=1607652) Agent delivered item, new score: 2
(MultiAgentEnvRunner pid=1607667) Agent delivered item, new score: 1
(MultiAgentEnvRunner pid=1607652) Agent delivered item, new score: 1
(MultiAgentEnvRunner pid=1607667) Agent delivered item, new score: 1
(MultiAgentEnvRunner pid=1607667) Agent delivered item, new score: 1
(MultiAgentEnvRunner pid=1607667) Agent delivered item, new score: 2
(MultiAgentEnvRunner pid=1607667) Agent delivered item, new score: 

In [6]:
# Verify observation space consistency
def verify_observation_space():
    test_env = env_creator({"reward_weights": reward_weights, "map_nr": 1})
    obs_space = test_env.observation_space("ai_rl_1")
    sample_obs = test_env.reset()[0]["ai_rl_1"]
    
    print(f"Observation space: {obs_space}")
    print(f"Sample observation shape: {sample_obs.shape}")
    print(f"Sample observation min/max: {np.min(sample_obs)}, {np.max(sample_obs)}")
    
    assert obs_space.contains(sample_obs), "Observation doesn't match observation space!"
    test_env.close()
    return obs_space

# Get verified observation space
obs_space = verify_observation_space()

Weighted rewards:

Agent 1: (0.5, 0.5)

Agent 2: (0.9, 0.1)

Observation space: Box(0.0, 50.0, (1668,), float32)
Sample observation shape: (1668,)
Sample observation min/max: 0.0, 50.0


In [7]:
# Define separate policies for each agent
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    return f"policy_{agent_id}"

In [8]:
# Configuration for multi-agent training
config = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True
    )
    .environment(
        env="spoiled_broth",
        env_config={
            "reward_weights": reward_weights,
            "map_nr": map_nr
        },
        clip_actions=True,
    )
    .multi_agent(
        policies={
            "policy_ai_rl_1": (
                None,  # Use default PPO policy
                env_creator({"map_nr": 1}).observation_space("ai_rl_1"),
                env_creator({"map_nr": 1}).action_space("ai_rl_1"),
                {}
            ),
            "policy_ai_rl_2": (
                None,  # Use default PPO policy
                env_creator({"map_nr": 1}).observation_space("ai_rl_2"),
                env_creator({"map_nr": 1}).action_space("ai_rl_2"),
                {}
            )
        },
        policy_mapping_fn=policy_mapping_fn,
        policies_to_train=["policy_ai_rl_1", "policy_ai_rl_2"]
    )
    .resources(num_gpus=1)  # Set to 1 if you have a GPU
    .env_runners(num_env_runners=2)
    .training(
        train_batch_size=4000,
        minibatch_size=500,
        num_epochs=10
    )
)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)



In [9]:
# Build algorithm with error handling
try:
    algo = config.build_algo()
    full_run_dir = algo.logdir
    NAME_RAY = Path(full_run_dir).name
    print(f"Algorithm built successfully: {NAME_RAY}")
except Exception as e:
    print(f"Error building algorithm: {e}")
    raise

2025-05-18 23:05:18,998	WARNING algorithm_config.py:4968 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html


2025-05-18 23:05:31,665	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
2025-05-18 23:05:34,294	INFO trainable.py:160 -- Trainable.setup took 15.265 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


Algorithm built successfully: PPO_spoiled_broth_2025-05-18_23-05-196iy07m5r


In [10]:
# Save training config
training_dir = Path(f"{base_save_dir}{NAME_RAY}")
training_dir.mkdir(parents=True, exist_ok=True)
shutil.copytree(full_run_dir, training_dir, dirs_exist_ok=True)

def safe_config_dict(d):
    def make_serializable(o):
        try:
            json.dumps(o)
            return o
        except TypeError:
            return str(o)

    return {
        k: make_serializable(v)
        for k, v in d.items()
    }

config_path = f"{training_dir}/config.txt"
with open(config_path, "w") as f:
    json.dump(safe_config_dict(config.to_dict()), f, indent=4)

for i in range(30): 
    result = algo.train()
    print(f"Iteration {i}:")
    print(f"  Agent 1 reward: {result['env_runners']['module_episode_returns_mean'].get('policy_ai_rl_1', 0)}")
    print(f"  Agent 2 reward: {result['env_runners']['module_episode_returns_mean'].get('policy_ai_rl_2', 0)}")
    print(f"  Total reward: {result['env_runners']['episode_return_mean']}")
    
    # Save checkpoint
    if i % 5 == 0:
        checkpoint_obtained = algo.save()
        checkpoint_path = Path(checkpoint_obtained.checkpoint.path)
        
        custom_checkpoint_dir = Path(f"{training_dir}/Checkpoint_{i}")
        custom_checkpoint_dir.mkdir(parents=True, exist_ok=True)

        for item in checkpoint_path.iterdir():
            dst = custom_checkpoint_dir / item.name
            if item.is_dir():
                shutil.copytree(item, dst, dirs_exist_ok=True)
            else:
                shutil.copy2(item, dst)

        print(f"Checkpoint {i} saved in {custom_checkpoint_dir}")
        
        #eval_result = algo.evaluate()
        #print(f"Evaluation reward: {eval_result['evaluation']['episode_reward_mean']}")

Iteration 0:
  Agent 1 reward: -36.35749999999997
  Agent 2 reward: -36.007500000000164
  Total reward: -72.36500000000014
Checkpoint 0 guardado en /data/samuel_lozano/cooked/saved_models/map_1/PPO_spoiled_broth_2025-05-18_23-05-196iy07m5r/Checkpoint_0
Iteration 1:
  Agent 1 reward: -40.492500000000014
  Agent 2 reward: -40.14583333333356
  Total reward: -80.63833333333355
Iteration 2:
  Agent 1 reward: -40.973541666666684
  Agent 2 reward: -40.151041666666885
  Total reward: -81.12458333333358
Iteration 3:
  Agent 1 reward: -41.67862500000001
  Agent 2 reward: -40.73412500000021
  Total reward: -82.41275000000023
Iteration 4:
  Agent 1 reward: -42.7663
  Agent 2 reward: -41.64270000000021
  Total reward: -84.40900000000022
Iteration 5:
  Agent 1 reward: -42.86519999999999
  Agent 2 reward: -41.463600000000184
  Total reward: -84.32880000000017
Checkpoint 5 guardado en /data/samuel_lozano/cooked/saved_models/map_1/PPO_spoiled_broth_2025-05-18_23-05-196iy07m5r/Checkpoint_5
Iteration 6:


2025-05-19 01:20:37,376	WARNING rollout_ops.py:121 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().


Iteration 23:
  Agent 1 reward: 10.386000000000054
  Agent 2 reward: 28.607599999999774
  Total reward: 38.99359999999983


2025-05-19 01:21:47,446	WARNING rollout_ops.py:121 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().


Iteration 24:
  Agent 1 reward: 10.386000000000054
  Agent 2 reward: 28.607599999999774
  Total reward: 38.99359999999983


KeyboardInterrupt: 

# Reload params

In [29]:
from ray.rllib.algorithms.algorithm import Algorithm
import torch

In [30]:
reload_map_nr = 1

training_id = "PPO_spoiled_broth_2025-05-18_22-22-47j6l54y_m"

checkpoint_nr = 0

In [34]:
# Restaura el algoritmo desde un checkpoint
reload_checkpoint_dir = f"/data/samuel_lozano/cooked/saved_models/map_{reload_map_nr}/{training_id}/Checkpoint_{checkpoint_nr}/"

reload_algo = Algorithm.from_checkpoint(reload_checkpoint_dir)

policy_module_1 = reload_algo.get_policy("policy_ai_rl_1")
policy_module_2 = reload_algo.get_policy("policy_ai_rl_2")

# Comparar pesos (si estás usando PyTorch)
params1 = policy_module_1.get_parameters()
params2 = policy_module_2.get_parameters()

2025-05-18 22:48:33,277	INFO trainable.py:160 -- Trainable.setup took 14.885 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


AttributeError: 'MultiAgentEnvRunner' object has no attribute 'get_policy'

In [ ]:
params1 = dict(policy_module_1.model.named_parameters())
params2 = dict(policy_module_2.model.named_parameters())

for name in params1:
    if name in params2:
        are_equal = torch.allclose(params1[name], params2[name])
        print(f"{name}: {'Equal' if are_equal else 'Different'}")

<bound method Deprecated.<locals>._inner.<locals>._ctor of PPO(env=spoiled_broth; env-runners=2; learners=0; multi-agent=True)>


In [ ]:
def params_to_vector(params):
    # Convierte diccionario de parámetros en un vector concatenado (PyTorch tensors)
    vectors = []
    for key, tensor in params.items():
        if isinstance(tensor, torch.Tensor):
            vectors.append(tensor.flatten())
        else:
            # Si no es tensor, intenta convertirlo o ignora
            pass
    return torch.cat(vectors)

v1 = params_to_vector(params1)
v2 = params_to_vector(params2)

diff = torch.norm(v1 - v2).item()
print(f"Diferencia entre parámetros de ambas políticas: {diff}")